In [1]:
import os

In [2]:
pwd

'd:\\AI ML Project\\oral-lesion-detection-ai\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'd:\\AI ML Project\\oral-lesion-detection-ai'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path
    training_data : Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH, Params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(Params_file_path)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Oral Cancer Image Dataset")

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir,
            training_data = Path(training_data)
        )

        return data_ingestion_config
     

In [ ]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
from PIL import Image, UnidentifiedImageError

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        """
        fetch data from url
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        zip_path = self.config.local_data_file
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



    def remove_unreadable_images(self):
        training_data = self.config.training_data
        deleted = 0
        total = 0

        for img_path in training_data.rglob("*.jpeg"):
            total += 1
            try:
                with Image.open(img_path) as img:
                    img.verify()
            except (UnidentifiedImageError, OSError) as e:
                try:
                    img_path.unlink()
                    deleted += 1
                except Exception as remove_err:
                    logger.error(f"Failed to remove {img_path}: {remove_err}")

        logger.info(f"Unreadable image removal complete — Total: {total}, Removed: {deleted}, Valid: {total - deleted}")
               
       



In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.remove_unreadable_images()

except Exception as e:
    raise e    

[2025-07-29 17:49:23,730: INFO: common: yaml file : config\config.yaml loaded succesfully]
[2025-07-29 17:49:23,734: INFO: common: yaml file : params.yaml loaded succesfully]
[2025-07-29 17:49:23,736: INFO: common: created directory at: artifacts]
[2025-07-29 17:49:23,738: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-29 17:49:23,740: INFO: 3514651227: Downloading data from https://drive.google.com/file/d/1kmrF8EWe8l3mHAkZoWujvA2paXeUiDxR/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1kmrF8EWe8l3mHAkZoWujvA2paXeUiDxR
From (redirected): https://drive.google.com/uc?export=download&id=1kmrF8EWe8l3mHAkZoWujvA2paXeUiDxR&confirm=t&uuid=b436c48d-75da-4bbf-a012-a9a60226f3d5
To: d:\AI ML Project\oral-lesion-detection-ai\artifacts\data_ingestion\data.zip
100%|██████████| 165M/165M [00:43<00:00, 3.80MB/s] 

[2025-07-29 17:50:10,190: INFO: 3514651227: Downloaded data from https://drive.google.com/file/d/1kmrF8EWe8l3mHAkZoWujvA2paXeUiDxR/view?usp=sharing into file artifacts/data_ingestion/data.zip]


[2025-07-29 17:50:13,305: WARNING: 3514651227: Removing unreadable image: artifacts\data_ingestion\Oral Cancer Image Dataset\CANCER\101.jpeg — cannot identify image file 'artifacts\\data_ingestion\\Oral Cancer Image Dataset\\CANCER\\101.jpeg']


c:\Anaconda\envs\oral\Lib\site-packages\PIL\Image.py:3570: UserWarning: image file could not be identified because AVIF support not installed
  warnings.warn(message)


[2025-07-29 17:50:14,200: WARNING: 3514651227: Removing unreadable image: artifacts\data_ingestion\Oral Cancer Image Dataset\CANCER\200.jpeg — cannot identify image file 'artifacts\\data_ingestion\\Oral Cancer Image Dataset\\CANCER\\200.jpeg']
[2025-07-29 17:50:14,732: WARNING: 3514651227: Removing unreadable image: artifacts\data_ingestion\Oral Cancer Image Dataset\CANCER\273.jpeg — cannot identify image file 'artifacts\\data_ingestion\\Oral Cancer Image Dataset\\CANCER\\273.jpeg']
[2025-07-29 17:50:15,052: WARNING: 3514651227: Removing unreadable image: artifacts\data_ingestion\Oral Cancer Image Dataset\CANCER\317.jpeg — cannot identify image file 'artifacts\\data_ingestion\\Oral Cancer Image Dataset\\CANCER\\317.jpeg']
[2025-07-29 17:50:15,137: WARNING: 3514651227: Removing unreadable image: artifacts\data_ingestion\Oral Cancer Image Dataset\CANCER\328.jpeg — cannot identify image file 'artifacts\\data_ingestion\\Oral Cancer Image Dataset\\CANCER\\328.jpeg']
[2025-07-29 17:50:15,751